# Initialization

Change to `valuenet` directory and add `src` path to `PYTHONPATH`.

In [1]:
%cd /home/ec2-user/SageMaker/valuenetforked

/home/ec2-user/SageMaker/valuenetforked


In [2]:
import sys
sys.path.insert(0, '/home/ec2-user/SageMaker/valuenetforked/src')
sys.path

['/home/ec2-user/SageMaker/valuenetforked/src',
 '/home/ec2-user/SageMaker/aws-sagemaker-notebook-valuenet',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/valuenet/lib/python38.zip',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/valuenet/lib/python3.8',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/valuenet/lib/python3.8/lib-dynload',
 '',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/valuenet/lib/python3.8/site-packages',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/valuenet/lib/python3.8/site-packages/IPython/extensions',
 '/home/ec2-user/.ipython']

Read environment

In [3]:
NER_API_SECRET=%env NER_API_SECRET
API_KEY=%env API_KEY
DB_USER=%env DB_USER
DB_PW=%env DB_PW
DB_HOST=%env DB_HOST
DB_PORT=%env DB_PORT
DB_SCHEMA="public"

# Train Model

In [4]:
# Empty gpu use
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [5]:
exp_name = "train-01"

**Results and final model**:
After training, you will be able to find the ground truth data, output of the model, evaluation results and intermediary data in the folder `valuenet/experiments/{exp_name}_date_time`.
If the accuracy was better than the last best accuracy, the model is also saved in the folder as `"best_model.pt"`.

**Note**: There will be errors of the type below. It is expected and normal.

    Exception
        Traceback (most recent call last):
            File "/home/ec2-user/SageMaker/valuenetforked/src/intermediate_representation/sem2sql/sem2SQL.py", line 691, in transform_semQL_to_sql
                result = transform(sem_ql_prediction[i], schemas[sem_ql_prediction[i]['db_id']])
            File "/home/ec2-user/SageMaker/valuenetforked/src/intermediate_representation/sem2sql/sem2SQL.py", line 246, in transform
        assert isinstance(c_instance, Root1)
    AssertionError

In [6]:
%run src/main.py --exp_name {exp_name} --data_set hack_zurich --cuda --batch_size 1 --num_epochs 5 --loss_epoch_threshold 70 --sketch_loss_weight 1.0 --beam_size 1 --seed 90

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: paulinemauryl (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


*** parsed configuration from command line and combine with constants ***
argument: cuda=True
argument: encoder_pretrained_model=facebook/bart-base
argument: max_seq_length=1024
argument: column_pointer=True
argument: embed_size=300
argument: hidden_size=300
argument: action_embed_size=128
argument: att_vec_size=300
argument: type_embed_size=128
argument: col_embed_size=300
argument: readout=identity
argument: column_att=affine
argument: dropout=0.3
argument: exp_name=train-01
argument: seed=90
argument: toy=False
argument: data_set=hack_zurich
argument: batch_size=1
argument: num_epochs=5.0
argument: lr_base=0.001
argument: lr_connection=0.0001
argument: lr_transformer=2e-05
argument: scheduler_gamma=0.5
argument: max_grad_norm=1.0
argument: clip_grad=5.0
argument: loss_epoch_threshold=70
argument: sketch_loss_weight=1.0
argument: beam_size=1
argument: decode_max_time_step=40
argument: data_dir=data/hack_zurich
argument: model_output_dir=experiments
Run experiment 'train-01__20210921_

Some weights of the model checkpoint at facebook/bart-base were not used when initializing BartModel: ['final_logits_bias']
- This IS expected if you are initializing BartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Successfully loaded pre-trained transformer 'facebook/bart-base'
Use Column Pointer:  True
Build optimizer and scheduler. Total training steps: 305.0
Start training with 5.0 epochs


  0%|          | 0/5 [00:12<?, ?it/s]

...

Not all values found during pre-processing for question "Which region had the lowest share of hybrid vehicles within the newly registered cars in the year 2012?". Replace values with dummy to make query fail



Evaluating:  75%|███████▌  | 12/16 [00:01<00:00,  8.82it/s]
                                             00,  8.85it/s]
Evaluating:  81%|████████▏ | 13/16 [00:01<00:00,  8.85it/s]
                                             00,  8.78it/s]
Evaluating:  94%|█████████▍| 15/16 [00:01<00:00,  8.65it/s]

Not all values found during pre-processing for question "How many municipalities have a share of electric vehicles within the newly registered cars higher than 0 per year?". Replace values with dummy to make query fail
Not all values found during pre-processing for question "How high is the total share of electric vehicles within the newly registered cars in the Canton Zurich in the year 2012?". Replace values with dummy to make query fail


                                             
100%|██████████| 5/5 [01:09<00:00, 13.91s/it]

Not all values found during pre-processing for question "Which region had the lowest share of electric vehicles within the newly registered cars in the year 2012?". Replace values with dummy to make query fail
in 0 times we found the correct results in another beam (failing queries: 16)
Epoch: 5    Sketch-Accuracy: 0.0     Accuracy: 0.0
Exception
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/valuenetforked/src/intermediate_representation/sem2sql/sem2SQL.py", line 691, in transform_semQL_to_sql
    result = transform(sem_ql_prediction[i], schemas[sem_ql_prediction[i]['db_id']])
  File "/home/ec2-user/SageMaker/valuenetforked/src/intermediate_representation/sem2sql/sem2SQL.py", line 246, in transform
    assert isinstance(c_instance, Root1)
AssertionError

How high is the total share of electric vehicles in the Canton Zurich in the year 2015?
SELECT T1.share_electric_cars FROM share_electric_cars AS T1         JOIN spatialunit AS T2 ON T1.spatialunit_id = T2.spatial